In [3]:
import pandas as pd
import numpy as np
import datetime

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Naive estimator

In [4]:
train = pd.read_csv('preprocessed_train.csv')
test = pd.read_csv('preprocessed_test.csv')

In [5]:
def naive_time_estimator(train, test):
    train['next position'] = train.groupby('case concept:name')['position'].shift(-1) #Creates new column with NaN values for the last even in a case
    test['next position'] = test.groupby('case concept:name')['position'].shift(-1)
    
    train = train.dropna(subset=['next position']) #Drop all last events per case
    
    train_position = train.groupby('position')['seconds_next'].mean()
    print(train_position)
    test['prediction_seconds'] = test.apply(lambda x: train_position[x['position']]
                                            if not pd.isnull(x['next position']) else np.nan, axis = 1)
    
    return test

naive_time_estimator(train, test)

position
1          0.585243
2         34.974184
3      10192.966870
4      12220.322218
5       1132.341762
           ...     
170       14.929000
171     1568.870000
172      304.951000
173        0.000000
174        3.406000
Name: seconds_next, Length: 174, dtype: float64


,Unnamed: 0,index,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,...,position,prev_event,2prev_event,next_event,prev_time,next_time,seconds_next,seconds_prev,next position,prediction_seconds
0,0,0,32422708117504,197219,2012-01-02 15:28:39.244,10000,112.0,A_SUBMITTED,COMPLETE,2012-01-02 15:28:39.244,...,1,FIRST EVENT,FIRST EVENT,A_PARTLYSUBMITTED,2012-01-02 15:28:39.244,2012-01-02 15:28:39.416,0.172,0.000,2.0,0.585243
1,1,1,32422708117505,197219,2012-01-02 15:28:39.244,10000,112.0,A_PARTLYSUBMITTED,COMPLETE,2012-01-02 15:28:39.416,...,2,A_SUBMITTED,FIRST EVENT,A_PREACCEPTED,2012-01-02 15:28:39.244,2012-01-02 15:29:09.496,30.080,0.172,3.0,34.974184
2,2,2,32422708117506,197219,2012-01-02 15:28:39.244,10000,112.0,A_PREACCEPTED,COMPLETE,2012-01-02 15:29:09.496,...,3,A_PARTLYSUBMITTED,A_SUBMITTED,W_Completeren aanvraag,2012-01-02 15:28:39.416,2012-01-02 15:29:09.906,0.410,30.080,4.0,10192.966870
3,3,3,32422708117507,197219,2012-01-02 15:28:39.244,10000,112.0,W_Completeren aanvraag,SCHEDULE,2012-01-02 15:29:09.906,...,4,A_PREACCEPTED,A_PARTLYSUBMITTED,W_Completeren aanvraag,2012-01-02 15:29:09.496,2012-01-02 19:41:59.956,15170.050,0.410,5.0,12220.322218
4,4,4,32422708117508,197219,2012-01-02 15:28:39.244,10000,10863.0,W_Completeren aanvraag,START,2012-01-02 19:41:59.956,...,5,W_Completeren aanvraag,A_PREACCEPTED,W_Completeren aanvraag,2012-01-02 15:29:09.906,2012-01-02 19:46:24.381,264.425,15170.050,6.0,1132.341762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22409,22409,14902,56199647068172,214373,2012-02-29 23:43:09.766,8500,10933.0,O_SENT,COMPLETE,2012-03-01 20:22:40.043,...,13,O_CREATED,O_SELECTED,W_Nabellen offertes,2012-03-01 20:22:40.016,2012-03-01 20:22:40.149,0.106,0.027,14.0,10493.881028
22410,22410,14903,56199647068173,214373,2012-02-29 23:43:09.766,8500,10933.0,W_Nabellen offertes,SCHEDULE,2012-03-01 20:22:40.149,...,14,O_SENT,O_CREATED,W_Completeren aanvraag,2012-03-01 20:22:40.043,2012-03-01 20:22:41.157,1.008,0.106,15.0,126701.952273
22411,22411,14904,56199647068174,214373,2012-02-29 23:43:09.766,8500,10933.0,W_Completeren aanvraag,COMPLETE,2012-03-01 20:22:41.157,...,15,W_Nabellen offertes,O_SENT,W_Nabellen offertes,2012-03-01 20:22:40.149,2012-03-10 12:46:22.700,750221.543,1.008,16.0,30880.625565
22412,22412,20144,56199647068175,214373,2012-02-29 23:43:09.766,8500,11119.0,W_Nabellen offertes,START,2012-03-10 12:46:22.700,...,16,W_Completeren aanvraag,W_Nabellen offertes,W_Nabellen offertes,2012-03-01 20:22:41.157,2012-03-10 12:50:54.881,272.181,750221.543,17.0,116338.299622


In [6]:
def MAE(test):
    test2 = test.dropna(subset=['prediction_seconds','seconds_next'])
    
    y_pred = test2['prediction_seconds']
    y_true = test2['seconds_next']
    
    return mean_absolute_error(y_true, y_pred)

In [7]:
MAE(test)

74668.08540604301